### 数据导入和情况

In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('data1.csv')
data.head()

,comment,star
0,口味：不知道是我口高了，还是这家真不怎么样。??我感觉口味确实很一般很一般。上菜相当快，我敢...,2
1,菜品丰富质量好，服务也不错！很喜欢！,4
2,说真的，不晓得有人排队的理由，香精香精香精香精，拜拜！,2
3,菜量实惠，上菜还算比较快，疙瘩汤喝出了秋日的暖意，烧茄子吃出了大阪烧的味道，想吃土豆片也是口...,5
4,先说我算是娜娜家风荷园开业就一直在这里吃??每次出去回来总想吃一回??有时觉得外面的西式简餐...,4


In [3]:
data['star'].unique()

array([2, 4, 5, 1], dtype=int64)

In [4]:
def make_label(star):
    if star > 3:
        return 1
    else:
        return 0
    
data['sentiment'] = data.star.apply(make_label)

In [5]:
data.head()

,comment,star,sentiment
0,口味：不知道是我口高了，还是这家真不怎么样。??我感觉口味确实很一般很一般。上菜相当快，我敢...,2,0
1,菜品丰富质量好，服务也不错！很喜欢！,4,1
2,说真的，不晓得有人排队的理由，香精香精香精香精，拜拜！,2,0
3,菜量实惠，上菜还算比较快，疙瘩汤喝出了秋日的暖意，烧茄子吃出了大阪烧的味道，想吃土豆片也是口...,5,1
4,先说我算是娜娜家风荷园开业就一直在这里吃??每次出去回来总想吃一回??有时觉得外面的西式简餐...,4,1


### SNOWNLP

In [6]:
from snownlp import SnowNLP

text1 = '这个东西不错'
text2 = '这个东西很垃圾'

s1 = SnowNLP(text1)
s2 = SnowNLP(text2)

print(s1.sentiments,s2.sentiments)

0.8623218777387431 0.21406279508712744


In [7]:
def snow_result(comemnt):
    s = SnowNLP(comemnt)
    if s.sentiments >= 0.6:
        return 1
    else:
        return 0
    
data['snlp_result'] = data.comment.apply(snow_result)

In [8]:
data.head(5)

,comment,star,sentiment,snlp_result
0,口味：不知道是我口高了，还是这家真不怎么样。??我感觉口味确实很一般很一般。上菜相当快，我敢...,2,0,0
1,菜品丰富质量好，服务也不错！很喜欢！,4,1,1
2,说真的，不晓得有人排队的理由，香精香精香精香精，拜拜！,2,0,1
3,菜量实惠，上菜还算比较快，疙瘩汤喝出了秋日的暖意，烧茄子吃出了大阪烧的味道，想吃土豆片也是口...,5,1,0
4,先说我算是娜娜家风荷园开业就一直在这里吃??每次出去回来总想吃一回??有时觉得外面的西式简餐...,4,1,0


In [9]:
counts = 0
for i in range(len(data)):
    if data.iloc[i,2] == data.iloc[i,3]:
        counts+=1

print(counts/len(data))

0.763


### 朴素贝叶斯
##### jieba分词

In [10]:
import jieba

def chinese_word_cut(mytext):
    return " ".join(jieba.cut(mytext))

data['cut_comment'] = data.comment.apply(chinese_word_cut)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\luopan\AppData\Local\Temp\jieba.cache
Loading model cost 0.566 seconds.
Prefix dict has been built succesfully.


In [11]:
data.head()

,comment,star,sentiment,snlp_result,cut_comment
0,口味：不知道是我口高了，还是这家真不怎么样。??我感觉口味确实很一般很一般。上菜相当快，我敢...,2,0,0,口味 ： 不 知道 是 我口 高 了 ， 还是 这家 真 不怎么样 。 ? ? 我 感觉 口...
1,菜品丰富质量好，服务也不错！很喜欢！,4,1,1,菜品 丰富 质量 好 ， 服务 也 不错 ！ 很 喜欢 ！
2,说真的，不晓得有人排队的理由，香精香精香精香精，拜拜！,2,0,1,说真的 ， 不 晓得 有人 排队 的 理由 ， 香精 香精 香精 香精 ， 拜拜 ！
3,菜量实惠，上菜还算比较快，疙瘩汤喝出了秋日的暖意，烧茄子吃出了大阪烧的味道，想吃土豆片也是口...,5,1,0,菜量 实惠 ， 上菜 还 算 比较 快 ， 疙瘩汤 喝出 了 秋日 的 暖意 ， 烧茄子 吃...
4,先说我算是娜娜家风荷园开业就一直在这里吃??每次出去回来总想吃一回??有时觉得外面的西式简餐...,4,1,0,先说 我 算是 娜娜 家风 荷园 开业 就 一直 在 这里 吃 ? ? 每次 出去 回来 总...


##### 划分数据集

In [12]:
X = data['cut_comment']
y = data.sentiment

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=22)

##### 词向量

In [13]:
from sklearn.feature_extraction.text import CountVectorizer

def get_custom_stopwords(stop_words_file):
    with open(stop_words_file) as f:
        stopwords = f.read()
    stopwords_list = stopwords.split('\n')
    custom_stopwords_list = [i for i in stopwords_list]
    return custom_stopwords_list

stop_words_file = '哈工大停用词表.txt'
stopwords = get_custom_stopwords(stop_words_file)

vect = CountVectorizer(max_df = 0.8, 
                       min_df = 3, 
                       token_pattern=u'(?u)\\b[^\\d\\W]\\w+\\b', 
                       stop_words=frozenset(stopwords))

In [14]:
test = pd.DataFrame(vect.fit_transform(X_train).toarray(), columns=vect.get_feature_names())
test.head()

,ipad,ok,ps,wifi,一下,一个个,一个半,一个多,一人,一份,...,麻烦,麻辣,麻酱,麻麻,黄瓜,黄盖,黄色,黏糊糊,黑椒,默默
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##### 训练模型

In [15]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

X_train_vect = vect.fit_transform(X_train)
nb.fit(X_train_vect, y_train)
train_score = nb.score(X_train_vect, y_train)
print(train_score)

0.899375


##### 测试模型

In [16]:
X_test_vect = vect.transform(X_test)
print(nb.score(X_test_vect, y_test))

0.8275


In [18]:
X_vec = vect.transform(X)
nb_result = nb.predict(X_vec)
data['nb_result'] = nb_result

In [20]:
data.head()

,comment,star,sentiment,snlp_result,cut_comment,nb_result
0,口味：不知道是我口高了，还是这家真不怎么样。??我感觉口味确实很一般很一般。上菜相当快，我敢...,2,0,0,口味 ： 不 知道 是 我口 高 了 ， 还是 这家 真 不怎么样 。 ? ? 我 感觉 口...,1
1,菜品丰富质量好，服务也不错！很喜欢！,4,1,1,菜品 丰富 质量 好 ， 服务 也 不错 ！ 很 喜欢 ！,1
2,说真的，不晓得有人排队的理由，香精香精香精香精，拜拜！,2,0,1,说真的 ， 不 晓得 有人 排队 的 理由 ， 香精 香精 香精 香精 ， 拜拜 ！,0
3,菜量实惠，上菜还算比较快，疙瘩汤喝出了秋日的暖意，烧茄子吃出了大阪烧的味道，想吃土豆片也是口...,5,1,0,菜量 实惠 ， 上菜 还 算 比较 快 ， 疙瘩汤 喝出 了 秋日 的 暖意 ， 烧茄子 吃...,1
4,先说我算是娜娜家风荷园开业就一直在这里吃??每次出去回来总想吃一回??有时觉得外面的西式简餐...,4,1,0,先说 我 算是 娜娜 家风 荷园 开业 就 一直 在 这里 吃 ? ? 每次 出去 回来 总...,0
